In [10]:
#Impporting all the libraries
import json
import matplotlib.pylab as plt
import pandas as pd
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
import numpy as np

In [11]:
#CReating a function for calculating F1 score from scratch
def confusion_matrix(actual,predicted): 
    x=np.zeros((2,2))
    for i in range(len(actual)):
        if int(actual[i])==-1 and int(predicted[i])==-1:
            x[0][0]+=1     #true negatives
        elif int(actual[i])==-1 and int(predicted[i])==1:
            x[0][1]+=1   #false positive
        elif int(actual[i])==1 and int(predicted[i])==-1:
            x[1][0]+=1    #false negaitve   
        elif int(actual[i])==1 and int(predicted[i])==1:
            x[1][1]+=1       #true positive
        
    return x


def calculate_f1(y_gold, y_model):
    conf_matrix= confusion_matrix(y_gold, y_model)
    precision= conf_matrix[1][1]/(conf_matrix[0][1]+conf_matrix[1][1])
    recall=conf_matrix[1][1]/(conf_matrix[1][0]+conf_matrix[1][1])
    f1= (2*precision*recall)/(precision+recall)
    return f1

In [12]:
class Classifier(object):

    def train(self, X, y):
        iterations = 10
        for iteration in range(iterations):
            for x_i, y_i in zip(X, y):
                self.process_example(x_i, y_i)
            
        self.finalize()

    def process_example(self, x, y):
        """
        Makes a prediction using the current parameter values for
        the features x and potentially updates the parameters based
        on the gradient. "x" is a dictionary which maps from the feature
        name to the feature value and y is either 1 or -1.
        """
        
        raise NotImplementedError

    def finalize(self):
        """Calculates the final parameter values for the averaged models."""
        pass

    def predict(self, X):
        """
        Predicts labels for all of the input examples.
        """
        y = []
        for x in X:
            y.append(self.predict_single(x))
        return y

    def predict_single(self, x):
        """
        Predicts a label, 1 or -1, for the input example. "x" is a dictionary
        which maps from the feature name to the feature value.
        """
        raise NotImplementedError

In [13]:
class Perceptron(Classifier):
    def __init__(self, features):
        """
        Initializes the parameters for the Perceptron model. "features"
        is a list of all of the features of the model where each is
        represented by a string.
        """
        # Do not change the names of these 3 data members 
        self.eta = 1
        self.w = {feature: 0.0 for feature in features}
        self.theta = 0

    def process_example(self, x, y):
        y_pred = self.predict_single(x)
        if y != y_pred:
            for feature, value in x.items():
                self.w[feature] += self.eta * y * value
            self.theta += self.eta * y

    def predict_single(self, x):
        score = 0
        for feature, value in x.items():
            score += self.w[feature] * value
        score += self.theta
        if score <= 0:
            return -1
        return 1

In [14]:
class AveragedPerceptron(Classifier):
    def __init__(self, features):
        self.eta = 1
        self.w = {feature: 0.0 for feature in features}                        #initialising weight
        self.averageweight={feature: 0.0 for feature in features}               #initialising average weight           
        self.theta = 0                                                          #initialising bias
        self.averagetheta=0                                                     #initialising average bias
        self.counter= 1                                                        #setting the counter to 1 initially
        # You will need to add data members here
        
    def process_example(self, x, y):
        y_pred = self.predict_single(x)
        if y != y_pred:
            for feature, value in x.items():
                self.w[feature] += self.eta * y * value                                 #updating the weights
                self.averageweight[feature] += self.eta * y * value*self.counter       #updating the average weights
            self.theta += self.eta * y                                                 #updating the bias
            self.averagetheta += self.eta * y*self.counter                             #updating the average bias
            self.counter+=1                                            #incremeting the counter irrespective of misclassification

        

    def predict_single(self, x):
        score = 0
        for feature, value in x.items():
            score += self.w[feature] * value
        score += self.theta
        if score <= 0:
            return -1
        return 1
        
        
        
    def finalize(self):
        for feature, weights in self.w.items():
            self.w[feature] = self.w[feature] - self.averageweight[feature]/self.counter     #updating the final averaged weight 
        self.theta = self.theta - self.averagetheta/self.counter                             #updating the final averaged bias
       

In [15]:
def load_ner_data(path):
    
#     Loads the NER data from a path (e.g. "ner/conll/train").
    # List of tuples for each sentence
    data = []
    for filename in os.listdir(path):
        with open(path + '/' + filename, 'r') as file:
            sentence = []
            for line in file:
                if line == '\n':
                    data.append(sentence)
                    sentence = []
                else:
                    sentence.append(tuple(line.split()))
    return data

In [16]:
def extract_ner_features_train(train):
    """
    Extracts feature dictionaries and labels from the data in "train"
    Additionally creates a list of all of the features which were created.
    """
    y = []
    X = []
    features = set()
    for sentence in train:
        padded = sentence[:]
        padded.insert(0, ('SSS', None))
        padded.append(('EEE', None))
        
        for i in range(1, len(padded) - 1):
            feats = []                                  #creating an empty list here which will be uodated with each condition
            y.append(1 if padded[i][1] == 'I' else -1)
            feat1 = 'w-1=' + str(padded[i - 1][0])
            feat2 = 'w+1=' + str(padded[i + 1][0])
            feats.append(feat1)
            feats.append(feat2)
            if i-2>=0:                                         #setting the conditions so that the list doesn't go out of bound
                feat3 = 'w-2=' + str(padded[i - 2][0])               
                feats.append(feat3)             
            if i+2<= len(padded)-1:                   #since the range in our condition is len-1 so i+2 should be less than that
                feat4 = 'w+2=' + str(padded[i + 2][0])
                feats.append(feat4)
            if i-3>=0:
                feat5 = 'w-3=' + str(padded[i - 3][0])
                feats.append(feat5)
            if i+3<= len(padded)-1:
                feat6 = 'w+3=' + str(padded[i + 3][0])
                feats.append(feat6)
            if i-1>=0 and i-2>=0:
                feat7 = 'w-1=' + str(padded[i - 1][0]) + "&w-2=" + str(padded[i - 2][0])
                feats.append(feat7)
            if i+2<= len(padded)-1 and i+1<= len(padded)-1:
                feat8 = 'w+1=' + str(padded[i + 1][0]) + "&w+2=" + str(padded[i + 2][0])
                feats.append(feat8)
            if i+1<= len(padded)-1 and i-1>=0:
                feat9 = 'w-1=' + str(padded[i - 1][0]) + "&w+1=" + str(padded[i + 1][0])
                feats.append(feat9)
            features.update(feats)
            feats = {feature: 1 for feature in feats}
            X.append(feats)
    return features, X, y

In [17]:
def extract_features_dev_or_test(data, features):
    """
    Extracts feature dictionaries and labels from "data". The only
    features which should be computed are those in "features".
    """
    y = []
    X = []
    for sentence in data:
        padded = sentence[:]
        padded.insert(0, ('SSS', None))
        padded.append(('EEE', None))
        
        for i in range(1, len(padded) - 1):
            feats=[]
            y.append(1 if padded[i][1] == 'I' else -1)
            feat1 = 'w-1=' + str(padded[i - 1][0])
            feat2 = 'w+1=' + str(padded[i + 1][0])
            feats.append(feat1)
            feats.append(feat2)
            if i-2>=0:
                feat3 = 'w-2=' + str(padded[i - 2][0])
                feats.append(feat3)
            if i+2<= len(padded) - 1:
                feat4 = 'w+2=' + str(padded[i + 2][0])
                feats.append(feat4)
            if i-3>=0:
                feat5 = 'w-3=' + str(padded[i - 3][0])
                feats.append(feat5)
            if i+3<= len(padded)-1:
                feat6 = 'w+3=' + str(padded[i + 3][0])
                feats.append(feat6)
            if i-1>=0 and i-2>=0:
                feat7 = 'w-1=' + str(padded[i - 1][0]) + "&w-2=" + str(padded[i - 2][0])
                feats.append(feat7)
            if i+2<= len(padded)-1 and i+1<= len(padded)-1:
                feat8 = 'w+1=' + str(padded[i + 1][0]) + "&w+2=" + str(padded[i + 2][0])
                feats.append(feat8)
            if i+1<= len(padded)-1 and i-1>=0:
                feat9 = 'w-1=' + str(padded[i - 1][0]) + "&w+1=" + str(padded[i + 1][0])
                feats.append(feat9)
            feats = {feature: 1 for feature in feats if feature in features}
            X.append(feats)
    return X, y

In [24]:
def run_ner_experiment(data_path):
    """
    Runs the NER experiment using the path to the ner data
    (e.g. "ner" from resources). 
    """
    train = load_ner_data(data_path + '/conll/train')
    conll_test = load_ner_data(data_path + '/conll/test')
    enron_test = load_ner_data(data_path + '/enron/test')

    features, X_train, y_train = extract_ner_features_train(train)
    X_conll_test, y_conll_test = extract_features_dev_or_test(conll_test, features)
    X_enron_test, y_enron_test = extract_features_dev_or_test(enron_test, features)
                 
    
    classifier = Perceptron(features)
    classifier.train(X_train, y_train)
    y_pred_conll = classifier.predict(X_conll_test)
    conll_f1_perceptron = calculate_f1(y_conll_test, y_pred_conll)
    y_pred_enron = classifier.predict(X_enron_test)
    enron_f1_perceptron = calculate_f1(y_enron_test, y_pred_enron)

    classifier = AveragedPerceptron(features)
    classifier.train(X_train, y_train)
    y_pred_conll = classifier.predict(X_conll_test)
    conll_f1_avgperceptron = calculate_f1(y_conll_test, y_pred_conll)
    y_pred_enron = classifier.predict(X_enron_test)
    enron_f1_avgperceptron = calculate_f1(y_enron_test, y_pred_enron)

    classifier = LinearSVC(loss='hinge')
    vectorizer = DictVectorizer()
    X_train_dict = vectorizer.fit_transform(X_train)
    X_conll_test_dict = vectorizer.transform(X_conll_test)     #Converting the conll and enron data to the required form 
    X_enron_test_dict = vectorizer.transform(X_enron_test)    
    classifier.fit(X_train_dict, y_train)
    y_pred_conll = classifier.predict(X_conll_test_dict)       #predicting for conll 
    conll_f1_svm = calculate_f1(y_conll_test, y_pred_conll)
    y_pred_enron = classifier.predict(X_enron_test_dict)
    enron_f1_svm = calculate_f1(y_enron_test, y_pred_enron)
    # to print the f1 score in a table, I will create a dataframe here
    data={"Algorithms":["Perceptron","Average Perceptron","SVM"], "ConLL_F1_score":[conll_f1_perceptron,conll_f1_avgperceptron,conll_f1_svm], "Enron_F1_score":[enron_f1_perceptron,enron_f1_avgperceptron,enron_f1_svm]}
    df=pd.DataFrame(data)
    print(df)

In [25]:
os.getcwd()                #getting the path of the file

'C:\\Users\\srish\\Downloads\\nlp assignment1\\released'

In [26]:
# Run the NER experiment. "ner" is the path to where the data is located.
features = run_ner_experiment('ner')

           Algorithms  ConLL_F1_score  Enron_F1_score
0          Perceptron        0.747623        0.200903
1  Average Perceptron        0.807854        0.218876
2                 SVM        0.827824        0.240682


### Comments on the result 

#### The three algorithms can be compared using the f1 scores obtained for the test data. We see that the f1 score for SVM algorithm is the highest for ConLL and Enron dataset. Thus it performs the best. It is better than perceptron based algortihms as it finds the best plane with the maximum margin and doesn't stop after correctly classifying data like perceptron. 

#### The f1 score decreased drastically for Enron data. One of the reasons could be the nature of the train and test data. The datasets come from entirely different backgrounds. Thus, there will be difference in the nature of the sentences on which we have trained our model(there will be new and unknown features for Enron data) . The other reason could be overfitting of the model on the train dataset which results in high score for similar test data and drastically low for data from other background. 
